In [13]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 2


In [15]:
import tensorflow as tf
print("GPUs:", tf.config.list_physical_devices('GPU'))


GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [19]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [20]:
import os
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping



In [21]:
import os
print(os.listdir("/kaggle/input"))


['fruit-and-vegetable-disease-healthy-vs-rotten']


In [22]:
import os
print(os.listdir("/kaggle/input/fruit-and-vegetable-disease-healthy-vs-rotten"))


['Fruit And Vegetable Diseases Dataset']


In [23]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 10
DATASET_DIR = "/kaggle/input/fruit-and-vegetable-disease-healthy-vs-rotten/Fruit And Vegetable Diseases Dataset"
MODEL_SAVE_PATH = "/kaggle/working/healthy_vs_rotten.h5"
CLASS_INDICES_PATH = "/kaggle/working/class_indices.json"



In [ ]:

def train_model():
    print("Checking dataset directories...")
    if not os.path.exists(DATASET_DIR):
        print(f"Error: Dataset directory '{DATASET_DIR}' not found.")
        print("Please download the dataset and extract it into a folder named 'dataset'.")
        return

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )
    
    if os.path.exists(os.path.join(DATASET_DIR, 'Train')):
        train_dir = os.path.join(DATASET_DIR, 'Train')
        test_dir = os.path.join(DATASET_DIR, 'Test')
        print(f"Found Train/Test structure in {DATASET_DIR}")
        
        train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='sparse' 
        )
        
        validation_generator = train_datagen.flow_from_directory(
            test_dir,
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='sparse'
        )
    else:
        print(f"Using flat structure in {DATASET_DIR} with validation split")
        train_generator = train_datagen.flow_from_directory(
            DATASET_DIR,
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='sparse',
            subset='training'
        )
        validation_generator = train_datagen.flow_from_directory(
            DATASET_DIR,
            target_size=IMG_SIZE,
            batch_size=BATCH_SIZE,
            class_mode='sparse',
            subset='validation'
        )

    num_classes = train_generator.num_classes
    print(f"Detected {num_classes} classes: {list(train_generator.class_indices.keys())}")
    
    # Save class indices
    with open(CLASS_INDICES_PATH, 'w') as f:
        json.dump(train_generator.class_indices, f)
    print(f"Saved class indices to {CLASS_INDICES_PATH}")

    # Model Building
    vgg = VGG16(input_shape=IMG_SIZE + (3,), weights='imagenet', include_top=False)

    for layer in vgg.layers:
        layer.trainable = False

    x = Flatten()(vgg.output)
    x = Dropout(0.5)(x)
    prediction = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=vgg.input, outputs=prediction)

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )

    model.summary()

    checkpoint = ModelCheckpoint(
        MODEL_SAVE_PATH,
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
    
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True
    )

    print("Starting training...")
    history = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=EPOCHS,
        steps_per_epoch=len(train_generator),
        validation_steps=len(validation_generator),
        callbacks=[checkpoint, early_stop]
    )
    print(f"Training complete. Model saved to {MODEL_SAVE_PATH}")

if __name__ == "__main__":
    train_model()
    

Checking dataset directories...
Using flat structure in /kaggle/input/fruit-and-vegetable-disease-healthy-vs-rotten/Fruit And Vegetable Diseases Dataset with validation split
Found 23431 images belonging to 28 classes.
Found 5846 images belonging to 28 classes.
Detected 28 classes: ['Apple__Healthy', 'Apple__Rotten', 'Banana__Healthy', 'Banana__Rotten', 'Bellpepper__Healthy', 'Bellpepper__Rotten', 'Carrot__Healthy', 'Carrot__Rotten', 'Cucumber__Healthy', 'Cucumber__Rotten', 'Grape__Healthy', 'Grape__Rotten', 'Guava__Healthy', 'Guava__Rotten', 'Jujube__Healthy', 'Jujube__Rotten', 'Mango__Healthy', 'Mango__Rotten', 'Orange__Healthy', 'Orange__Rotten', 'Pomegranate__Healthy', 'Pomegranate__Rotten', 'Potato__Healthy', 'Potato__Rotten', 'Strawberry__Healthy', 'Strawberry__Rotten', 'Tomato__Healthy', 'Tomato__Rotten']
Saved class indices to /kaggle/working/class_indices.json


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 28)             │       702,492 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,417,180 (58.81 MB)

 Trainable params: 702,492 (2.68 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Starting training...
Epoch 1/10
 231/1465 ━━━━━━━━━━━━━━━━━━━━ 5:39 275ms/step - accuracy: 0.3878 - loss: 2.4793

In [ ]:

/kaggle/working/healthy_vs_rotten.h5


In [13]:
import tensorflow as tf
import json
import numpy as np
from tensorflow.keras.preprocessing import image

# Load trained model
model = tf.keras.models.load_model("/kaggle/working/healthy_vs_rotten.h5")

# Load class indices
with open("/kaggle/working/class_indices.json", "r") as f:
    class_indices = json.load(f)

# Reverse dictionary (number → class name)
idx_to_class = {v: k for k, v in class_indices.items()}

print("Classes:", idx_to_class)


Classes: {0: 'Apple__Healthy', 1: 'Apple__Rotten', 2: 'Banana__Healthy', 3: 'Banana__Rotten', 4: 'Bellpepper__Healthy', 5: 'Bellpepper__Rotten', 6: 'Carrot__Healthy', 7: 'Carrot__Rotten', 8: 'Cucumber__Healthy', 9: 'Cucumber__Rotten', 10: 'Grape__Healthy', 11: 'Grape__Rotten', 12: 'Guava__Healthy', 13: 'Guava__Rotten', 14: 'Jujube__Healthy', 15: 'Jujube__Rotten', 16: 'Mango__Healthy', 17: 'Mango__Rotten', 18: 'Orange__Healthy', 19: 'Orange__Rotten', 20: 'Pomegranate__Healthy', 21: 'Pomegranate__Rotten', 22: 'Potato__Healthy', 23: 'Potato__Rotten', 24: 'Strawberry__Healthy', 25: 'Strawberry__Rotten', 26: 'Tomato__Healthy', 27: 'Tomato__Rotten'}


In [1]:
img_path = "/kaggle/input/fruit-and-vegetable-disease-healthy-vs-rotten/Fruit And Vegetable Diseases Dataset/Apple__Healthy/FreshApple (1).jpg"


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image

IMG_SIZE = (224, 224)

img_path = "/kaggle/input/fruit-and-vegetable-disease-healthy-vs-rotten/Fruit And Vegetable Diseases Dataset/Apple__Healthy/FreshApple (1).jpg"

# Load image
img = image.load_img(img_path, target_size=IMG_SIZE)
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
pred = model.predict(img_array)

pred_index = np.argmax(pred)
pred_label = idx_to_class[pred_index]
confidence = float(np.max(pred))

# Display
plt.imshow(img)
plt.axis("off")
plt.title(f"{pred_label} | Confidence: {confidence:.2f}")
plt.show()

print("Prediction:", pred_label)
print("Confidence:", confidence)


NameError: name 'model' is not defined